In [2]:
import xarray as xr
import geopandas as gpd
import rasterio
from rasterio.mask import mask
import numpy as np

# Function to clip the NetCDF file using the shapefile
def clip_netcdf_with_shapefile(nc_file, shapefile, output_nc_file):
    # Load the NetCDF data
    ds = xr.open_dataset(nc_file)

    # Load the shapefile
    gdf = gpd.read_file(shapefile)
    
    # Ensure the shapefile is in the same coordinate reference system (CRS) as the NetCDF data
    if 'crs' in ds.attrs:
        crs = ds.attrs['crs']
        gdf = gdf.to_crs(crs)
    
    # Extract geometry from shapefile
    shapes = [feature["geometry"] for feature in gdf.__geo_interface__["features"]]

    # Initialize a list to store the clipped data
    clipped_data = []
    
    # Iterate over variables in the NetCDF file
    for var in ds.data_vars:
        # Get the data array for the variable
        data_array = ds[var]

        #
        print(data_array)
        
        # Ensure the data array has spatial dimensions (lat, lon)
        if 'lat' in data_array.dims and 'lon' in data_array.dims:
            # Create a mask using rasterio
            with rasterio.open(nc_file) as src:
                out_image, out_transform = mask(src, shapes, crop=True)
                out_meta = src.meta.copy()
                
            # Update the metadata to match the shape of the clipped data
            out_meta.update({"height": out_image.shape[1],
                             "width": out_image.shape[2],
                             "transform": out_transform})
            
            # Convert the clipped data to an xarray DataArray
            clipped_array = xr.DataArray(out_image[0], coords={'lat': data_array['lat'], 'lon': data_array['lon']}, dims=['lat', 'lon'])
            
            # Append the clipped data array to the list
            clipped_data.append(clipped_array)
    
    # Combine the clipped data arrays into a single dataset
    clipped_ds = xr.Dataset({var: clipped_data[i] for i, var in enumerate(ds.data_vars)})
    
    # Save the clipped dataset to a new NetCDF file
    clipped_ds.to_netcdf(output_nc_file)
    
    return clipped_ds

# Example usage
nc_file = r'D:\Git\Projects\Runtime files\chirps20GlobalDailyP05_31cf_9fff_f151.nc'
shapefile = 'D:\Git\Projects\Runtime files\LV_Kagera_Akanyaru.shp'
output_nc_file = 'output_clipped_file.nc'

clipped_ds = clip_netcdf_with_shapefile(nc_file, shapefile, output_nc_file)
print(clipped_ds)


<xarray.DataArray 'precip' (time: 276, latitude: 39, longitude: 36)>
[387504 values with dtype=float32]
Coordinates:
  * time       (time) datetime64[ns] 2023-04-30 2023-05-01 ... 2024-01-30
  * latitude   (latitude) float32 -3.075 -3.025 -2.975 ... -1.275 -1.225 -1.175
  * longitude  (longitude) float32 29.18 29.22 29.27 29.32 ... 30.82 30.88 30.93
Attributes:
    colorBarMaximum:  25.0
    colorBarMinimum:  0.0
    ioos_category:    Meteorology
    long_name:        Precipitation
    standard_name:    lwe_precipitation_rate
    time_step:        day
    units:            mm/day


IndexError: list index out of range

In [3]:
import xarray as xr
import geopandas as gpd
import rasterio
from rasterio.mask import mask
import numpy as np

def clip_netcdf_with_shapefile(nc_file, shapefile, output_nc_file):
    # Load the NetCDF data
    ds = xr.open_dataset(nc_file)

    # Load the shapefile
    gdf = gpd.read_file(shapefile)
    
    # Ensure the shapefile is in the same coordinate reference system (CRS) as the NetCDF data
    if 'crs' in ds.attrs:
        crs = ds.attrs['crs']
        gdf = gdf.to_crs(crs)
    
    # Extract geometry from shapefile
    shapes = [feature["geometry"] for feature in gdf.__geo_interface__["features"]]

    clipped_data = {}
    
    # Iterate over variables in the NetCDF file
    for var in ds.data_vars:
        data_array = ds[var]
        
        if 'lat' in data_array.dims and 'lon' in data_array.dims:
            # Convert xarray DataArray to a format rasterio can handle
            lat = data_array['lat'].values
            lon = data_array['lon'].values
            values = data_array.values
            
            # Define affine transform for the data array
            transform = rasterio.transform.from_origin(
                lon.min(), lat.max(), np.abs(lon[1] - lon[0]), np.abs(lat[1] - lat[0])
            )
            
            # Create a rasterio in-memory dataset
            with rasterio.MemoryFile() as memfile:
                with memfile.open(
                    driver='GTiff',
                    height=values.shape[0],
                    width=values.shape[1],
                    count=1,
                    dtype=values.dtype,
                    crs='+proj=latlong',
                    transform=transform,
                ) as dataset:
                    dataset.write(values, 1)
                    out_image, out_transform = mask(dataset, shapes, crop=True)
                    
                    # Convert the clipped data to xarray DataArray
                    clipped_array = xr.DataArray(
                        out_image[0],
                        coords={
                            'lat': np.arange(out_image.shape[1]),
                            'lon': np.arange(out_image.shape[2])
                        },
                        dims=['lat', 'lon']
                    )
                    
                    clipped_data[var] = clipped_array
    
    # Combine the clipped data arrays into a single dataset
    clipped_ds = xr.Dataset(clipped_data)
    
    # Save the clipped dataset to a new NetCDF file
    clipped_ds.to_netcdf(output_nc_file)
    
    return clipped_ds

# Example usage
nc_file = r'D:\Git\Projects\Runtime files\chirps20GlobalDailyP05_31cf_9fff_f151.nc'
shapefile = 'D:\Git\Projects\Runtime files\LV_Kagera_Akanyaru.shp'
output_nc_file = 'output_clipped_file.nc'

clipped_ds = clip_netcdf_with_shapefile(nc_file, shapefile, output_nc_file)
print(clipped_ds)


<xarray.Dataset>
Dimensions:  ()
Data variables:
    *empty*


In [4]:
import xarray as xr
import geopandas as gpd
import rasterio
from rasterio.mask import mask
import numpy as np

def clip_netcdf_with_shapefile(nc_file, shapefile, output_nc_file):
    # Load the NetCDF data
    ds = xr.open_dataset(nc_file)

    # Load the shapefile
    gdf = gpd.read_file(shapefile)
    
    # Ensure the shapefile is in the same coordinate reference system (CRS) as the NetCDF data
    if 'crs' in ds.attrs:
        crs = ds.attrs['crs']
        gdf = gdf.to_crs(crs)
    else:
        print("No CRS information in NetCDF file. Assuming the shapefile is already in the correct CRS.")

    # Extract geometry from shapefile
    shapes = [feature["geometry"] for feature in gdf.__geo_interface__["features"]]

    clipped_data = {}

    # Iterate over variables in the NetCDF file
    for var in ds.data_vars:
        data_array = ds[var]
        
        if 'lat' in data_array.dims and 'lon' in data_array.dims:
            # Ensure lat and lon are sorted ascendingly
            if data_array.lat[0] > data_array.lat[-1]:
                data_array = data_array.reindex(lat=list(reversed(data_array.lat)))
            if data_array.lon[0] > data_array.lon[-1]:
                data_array = data_array.reindex(lon=list(reversed(data_array.lon)))
                
            lat = data_array['lat'].values
            lon = data_array['lon'].values
            values = data_array.values

            # Define affine transform for the data array
            transform = rasterio.transform.from_origin(
                lon.min(), lat.max(), np.abs(lon[1] - lon[0]), np.abs(lat[1] - lat[0])
            )
            
            # Create a rasterio in-memory dataset
            with rasterio.MemoryFile() as memfile:
                with memfile.open(
                    driver='GTiff',
                    height=values.shape[0],
                    width=values.shape[1],
                    count=1,
                    dtype=values.dtype,
                    crs='+proj=latlong',
                    transform=transform,
                ) as dataset:
                    dataset.write(values, 1)
                    out_image, out_transform = mask(dataset, shapes, crop=True)
                    
                    # Generate new coordinates for the clipped array
                    new_lon = np.linspace(out_transform.c, out_transform.c + out_transform.a * out_image.shape[2], out_image.shape[2])
                    new_lat = np.linspace(out_transform.f, out_transform.f + out_transform.e * out_image.shape[1], out_image.shape[1])

                    # Convert the clipped data to xarray DataArray
                    clipped_array = xr.DataArray(
                        out_image[0],
                        coords={
                            'lat': new_lat,
                            'lon': new_lon
                        },
                        dims=['lat', 'lon']
                    )
                    
                    clipped_data[var] = clipped_array
    
    if clipped_data:
        # Combine the clipped data arrays into a single dataset
        clipped_ds = xr.Dataset(clipped_data)
        
        # Save the clipped dataset to a new NetCDF file
        clipped_ds.to_netcdf(output_nc_file)
    else:
        print("No variables with 'lat' and 'lon' dimensions were found and clipped.")
        clipped_ds = xr.Dataset()

    return clipped_ds

# Example usage
nc_file = r'D:\Git\Projects\Runtime files\chirps20GlobalDailyP05_31cf_9fff_f151.nc'
shapefile = 'D:\Git\Projects\Runtime files\LV_Kagera_Akanyaru.shp'
output_nc_file = 'output_clipped_file.nc'

clipped_ds = clip_netcdf_with_shapefile(nc_file, shapefile, output_nc_file)
print(clipped_ds)


No CRS information in NetCDF file. Assuming the shapefile is already in the correct CRS.
No variables with 'lat' and 'lon' dimensions were found and clipped.
<xarray.Dataset>
Dimensions:  ()
Data variables:
    *empty*
